In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

modify_phi_tokenizer_dir = "phi-1"
modify_phi_tokenizer = AutoTokenizer.from_pretrained(modify_phi_tokenizer_dir)

modify_phi_tokenizer.encode("<image>")

/home/yujiaxi/anaconda3/envs/LLM/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[50295]

In [2]:
len(modify_phi_tokenizer)

50296

In [3]:
phi_model = AutoModelForCausalLM.from_pretrained(
    modify_phi_tokenizer_dir, device_map="cuda:0", torch_dtype=torch.bfloat16, trust_remote_code=True
)

In [4]:
(phi_model.model.embed_tokens,)

(Embedding(51200, 2048),)

In [5]:
phi_model.lm_head

Linear(in_features=2048, out_features=51200, bias=True)

In [2]:
clip_model_name_or_path = (
    "openai/clip-vit-large-patch14-336"
)
phi_model_name_or_path = "phi-1"

In [3]:
from transformers import AutoModel, AutoModelForCausalLM, AutoTokenizer, AutoProcessor

clip_model = AutoModel.from_pretrained(clip_model_name_or_path, device_map="cuda:0")
llm_model = AutoModelForCausalLM.from_pretrained(
    phi_model_name_or_path, device_map="cuda:0"
)

In [5]:
llm_tokenizer = AutoTokenizer.from_pretrained(phi_model_name_or_path)
llm_tokenizer.encode("<image>")

[50295]

In [6]:
from transformers import LlavaForConditionalGeneration, LlavaConfig

In [7]:
# Initializing a CLIP-vision config
vision_config = clip_model.vision_model.config

# Initializing a llm config
text_config = llm_model.config

# Initializing a llava-1.5-7b style configuration
configuration = LlavaConfig(vision_config, text_config)

# Initializing a model from the llava-1.5-7b style configuration
model = LlavaForConditionalGeneration(configuration)

In [8]:
model.vision_tower.vision_model.embeddings

CLIPVisionEmbeddings(
  (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
  (position_embedding): Embedding(577, 1024)
)

In [9]:
clip_model.vision_model.embeddings

CLIPVisionEmbeddings(
  (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
  (position_embedding): Embedding(577, 1024)
)

In [10]:
model.vision_tower.vision_model = clip_model.vision_model

In [11]:
model.language_model = llm_model

In [12]:
llm_model.model.embed_tokens.weight.data[:, :2]

tensor([[-0.0242, -0.0162],
        [ 0.0093,  0.0236],
        [-0.0122,  0.0200],
        ...,
        [-0.0016,  0.0031],
        [-0.0008, -0.0027],
        [ 0.0003, -0.0019]], device='cuda:0')

In [13]:
model.language_model.model.embed_tokens.weight.data[:, :2]

tensor([[-0.0242, -0.0162],
        [ 0.0093,  0.0236],
        [-0.0122,  0.0200],
        ...,
        [-0.0016,  0.0031],
        [-0.0008, -0.0027],
        [ 0.0003, -0.0019]], device='cuda:0')

In [14]:
model.config.pad_token_id

In [15]:
model.config.pad_token_id = llm_tokenizer.pad_token_id
model.config.pad_token_id

In [16]:
model.config.image_token_index

32000

In [17]:
llm_tokenizer.encode("<image>")[0]

50295

In [18]:
model.config.image_token_index = llm_tokenizer.encode("<image>")[0]
model.config.image_token_index

50295

In [19]:
model.save_pretrained("/home/yujiaxi/DLModels/llavalearn/phi_model/model001")

[2024-12-05 18:05:00,412] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/yujiaxi/anaconda3/envs/LLM/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/yujiaxi/anaconda3/envs/LLM/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/yujiaxi/anaconda3/envs/LLM/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/yujiaxi/anaconda3/envs/LLM/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/yujiaxi/anaconda3/envs/LLM/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/yujiaxi/anaconda3/envs/LLM/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/home/yujiaxi/anaconda3/envs/LLM/compiler_compat/ld: /u

In [21]:
llm_tokenizer.save_pretrained("phi_model/model001")

('phi_model/model001/tokenizer_config.json',
 'phi_model/model001/special_tokens_map.json',
 'phi_model/model001/vocab.json',
 'phi_model/model001/merges.txt',
 'phi_model/model001/added_tokens.json',
 'phi_model/model001/tokenizer.json')

In [22]:
autoprocessor = AutoProcessor.from_pretrained(clip_model_name_or_path)
autoprocessor.save_pretrained("/home/yujiaxi/DLModels/llavalearn/phi_model/model002")

[]

In [ ]:
from transformers import LlavaProcessor, LlavaForConditionalGeneration
import torch

model_name_or_path = "phi_model/model001"

llava_processor = LlavaProcessor.from_pretrained(model_name_or_path)
model = LlavaForConditionalGeneration.from_pretrained(
    model_name_or_path, device_map="cuda:0", torch_dtype=torch.bfloat16
)

/home/yujiaxi/anaconda3/envs/LLM/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from PIL import Image

prompt_text = "<image>\nWhat are these?"

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt_text},
]
prompt = llava_processor.tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

image_path = "1000_F_714460814_r331zxVydZFeuFk7swM3ILo9xEwRM5px.jpg"
image = Image.open(fp=image_path)

inputs = llava_processor(text=prompt, images=image, return_tensors="pt")

for tk in inputs.keys():
    inputs[tk] = inputs[tk].to(model.device)
generate_ids = model.generate(**inputs, max_new_tokens=50)
gen_text = llava_processor.batch_decode(
    generate_ids, skip_special_tokens=False, clean_up_tokenization_spaces=False
)[0]

print(gen_text)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
<image>
What are these?<|im_end|>
<|im_start|>assistant
Assistant
Assistant
Assistant
Assistant
Assistant
Assistant
Assistant
Assistant
Assistant
Assistant
Assistant
Assistant
Assistant
Assistant
Assistant
Assistant
Assistant
Assistant
Assistant
Assistant
Assistant
Assistant
Assistant
Assistant
Assistant

